In [1]:
import gradio as gr
from transformers import pipeline
import numpy as np
import openai
import warnings
warnings.filterwarnings('ignore')
import os
from dotenv import load_dotenv
load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")


C:\Users\c_mar\anaconda3\envs\dev\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
# Preference
# Initialize the ASR pipeline
transcriber = pipeline("automatic-speech-recognition", model="openai/whisper-base.en")

def transcribe_and_save(audio):
    sr, y = audio
    y = y.astype(np.float32)
    y /= np.max(np.abs(y))
    transcription = transcriber({"sampling_rate": sr, "raw": y})["text"]
    
    # Write the transcription to a file
    with open("transcription_movie_selection.txt", "w") as file:
        file.write(transcription)
    
    return transcription

# Create the Gradio interface
demo = gr.Interface(
    transcribe_and_save,
    gr.Audio(sources=["microphone"]),
    "text",
)

demo.launch(share=False)
#change to True for outside URL


Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.


In [4]:
#pure movie selection analysis with OpenAI
#preprocess txt file

with open('transcription_movie_selection.txt', 'r', encoding="utf-8") as file:
    text = file.read()


In [5]:
# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

# Set your OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")

# Function time

def get_preference(question):
    messages = [
        {"role": "system", "content": "Determine three movies to make the person happier based on the provided text."},
        {"role": "user", "content": question}
    ]
    
    # Use the chat completions method
    response = openai.chat.completions.create(
        model="gpt-4",
        messages=messages
    )
    
    # Extract the answer from the response
    answer = response.choices[0].message.content
    return answer

def analyze_document(filename):
    with open(filename, 'r') as file:
        text = file.read()
    preference = get_preference(text)
    print(f"Movie Recommendations: {preference}")

analyze_document('transcription_movie_selection.txt')


Movie Recommendations: 1. "You've Got Mail" - While not specifically about Stanley Mugs and strong coffee, this film definitely focuses a lot on the daily coffee shop experience and uses it as a backdrop for the central romance. 

2. "Groundhog Day" - In this classic comedy, the lead character played by Bill Murray wakes up to the same day over and over again, always starting his day with a cup of coffee.

3. "Coffee and Cigarettes" -  This is an anthology film composed of short stories linked by the twin themes, coffee and cigarettes. The stories are all conversations taking place in coffee shops and they involve a variety of quirky, interesting characters.
